## Simple notebook for testing language model as a way of measuring implicit language style of guidelines vs. community posts and comments
Currently using the no country for old members idea of using snapshot language models, right now just training one language models on all the posts over less than a month then compare to a held out subset, the community guidelines, and other community guidelines 

In [11]:
import numpy as np
import csv as csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import math
import json
from datetime import datetime
from termcolor import colored
import os
from sklearn.model_selection import train_test_split
from langdetect import detect

from nltk.util import bigrams, ngrams
from nltk.lm.preprocessing import pad_both_ends, flatten, padded_everygram_pipeline
from nltk.lm.smoothing import GoodTuring
from nltk.tokenize import word_tokenize
from nltk.lm.models import InterpolatedLanguageModel, WittenBellInterpolated, KneserNeyInterpolated, Laplace
from nltk.lm.api import LanguageModel
from nltk.lm import NgramCounter

os.chdir('../data/test_subs_2017/')


In [12]:
df_top_posts = pd.read_csv('../cleaned/top_posts_cleaned.csv')
df_subs = pd.read_csv('../cleaned/subbreddits.csv')

In [4]:
def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False
    
def make_text(df, text_col):
    full_text = df[text_col].apply(word_tokenize).values
    return list(full_text)


In [5]:

# define function for chunking text in a certian size, so you don't just throw out everything above 'k'\
# will still return only things that are exactly k
# currently not using this
def chunks(l, k):
    n = max(1, k)
    chunked_list = (l[i:i+n] for i in range(0, len(l), n))
    return [c for c in chunked_list if len(c) == k]


def entropy(lm, text):
    try:
        return lm.entropy(text)
    except:
        return -1
    
    

def make_test_text(df, col):
    test_top_posts_text = make_text(df, col)
    test_top_posts_text_threshold = list(filter(lambda x: len(x) > k, test_top_posts_text))
    test_top_posts_text_threshold = list(map(lambda x: list(ngrams(x[:k], 2)), test_top_posts_text_threshold))
    return test_top_posts_text_threshold


In [1]:
sci_text_dec

NameError: name 'sci_text_dec' is not defined

In [21]:
sci_text_dec = make_text(df_text_science_dec.fillna(' '), 'body')


In [9]:
### collect the science data
df_comments_science = pd.read_csv('../cleaned/science_comments_2017.csv')
# df_posts_science = pd.read_csv('../cleaned/science_posts_2017.csv')
print('done collecting comments')


# rename fulltext as body
df_posts_science = df_posts_science.rename(index=str, columns={"fulltext": "body"})
# append all comments and post together
df_text_science = df_comments_science[['body', 'created_month', 'subreddit', 'author']].append(df_posts_science[['body', 'created_month', 'subreddit', 'author']])
print('done putting posts/comments together')

### select from specific month (one for now)
mon = 12.0
df_text_science_dec = df_text_science[df_text_science['created_month'] == mon]
print('selected specific month ', mon)

### get only english posts

### break everything into tokens
sci_text_dec = make_text(df_text_science_dec.fillna(' '), 'body')
print('done tokenizing')

### ignore everything shorter than k = 20 words
k = 20
# sci_text_dec_threshold = list(filter(lambda x: len(x) > k, sci_text_dec))
# print('done cutting at', k, ' words')

### split into test and train
dev_size = 0.1
train_dec, dev_dec = train_test_split(sci_text_dec, test_size=dev_size, random_state=1)
print('done splitting at ', dev_size)


# set up train vocab - TODO: might need to go before the split 
train, vocab = padded_everygram_pipeline(2, train_dec)
lm = KneserNeyInterpolated(2)
lm.fit(train, vocab)
print('done training')

# get cross entropy of sampled 100 posts/comments, get avg, each comment/post must be 50 words

# import guidelines (for all subreddits)

# import top posts (from all subreddits)



done collecting posts/comments
done putting posts/comments together
selected specific month  12.0
done tokenizing
done splitting at  0.1
done training


In [13]:
k = 20

test = list(map(lambda x: list(ngrams(x[:k], 2)), dev_dec))


In [7]:
### 

df_comments_science = pd.read_csv('../cleaned/science_comments_2017.csv')
### select from specific month (one for now)
mon = 12.0
df_comments_science_dec = df_comments_science[df_comments_science['created_month'] == mon]
print('selected specific month ', mon)

### get only english posts

### break everything into tokens
sci_text_dec = make_text(df_comments_science_dec.fillna(' '), 'body')
print('done tokenizing')



selected specific month  12.0
done tokenizing


NameError: name 'train_dec' is not defined

In [9]:
from nltk.lm.smoothing import GoodTuring

WittenBellInterpolated(2, smoothing_cls=GoodTuring)

TypeError: __init__() got multiple values for argument 'smoothing_cls'

In [10]:
# set up train vocab - TODO: might need to go before the split 
# train, vocab = padded_everygram_pipeline(2, sci_text_dec)
lm = WittenBellInterpolated(2)
lm.fit(train, vocab)

In [14]:
df_top_posts['body'] = df_top_posts['title'].fillna(' ') + df_top_posts['selftext'].fillna(' ')

In [23]:
other

[[('Hi', '!'),
  ('!', 'We'),
  ('We', "'re"),
  ("'re", 'from'),
  ('from', 'The'),
  ('The', 'Independent'),
  ('Independent', "'s"),
  ("'s", 'US'),
  ('US', 'team'),
  ('team', ','),
  (',', 'including'),
  ('including', 'International'),
  ('International', 'Editor'),
  ('Editor', 'Chris'),
  ('Chris', 'Stevenson'),
  ('Stevenson', 'and'),
  ('and', 'reporters'),
  ('reporters', 'Clark'),
  ('Clark', 'Mindock')],
 [('Friday', 'Fun'),
  ('Fun', 'Thread'),
  ('Thread', '!'),
  ('!', 'It'),
  ('It', "'s"),
  ("'s", 'finally'),
  ('finally', 'Friday'),
  ('Friday', '!'),
  ('!', 'That'),
  ('That', 'means'),
  ('means', 'it'),
  ('it', "'s"),
  ("'s", 'time'),
  ('time', 'to'),
  ('to', 'sit'),
  ('sit', 'back'),
  ('back', ','),
  (',', 'have'),
  ('have', 'beverage')],
 [('Judge', 'Tells'),
  ('Tells', 'Roger'),
  ('Roger', 'Stone'),
  ('Stone', 'His'),
  ('His', 'Case'),
  ('Case', 'Is'),
  ('Is', 'Not'),
  ('Not', '‘'),
  ('‘', 'A'),
  ('A', 'Book'),
  ('Book', 'Tour'),
  ('Tour',

In [13]:
df_top_posts_science = df_top_posts[df_top_posts['subreddit_name'] == 'science']
df_top_posts_other = df_top_posts[df_top_posts['subreddit_name'] != 'science']

sci = make_test_text(df_top_posts_science, 'body')
other = make_test_text(df_top_posts_other, 'body')


KeyError: 'body'

In [16]:
sci_rules = make_test_text(df_subs[df_subs['subreddit_name'] == 'science'], 'public_descr')
other_rules = make_test_text(df_subs[df_subs['subreddit_name'] != 'science'], 'public_descr')

In [167]:
other_rule_ent = list(map(lambda x: entropy(lm, x), other_rules))
print(sum(filter(lambda x: x != -1, other_ent)) / len(list(filter(lambda x: x != -1, other_ent))))

7.717695658686901


In [ ]:
entropy(lm, other_rules[1])

In [168]:
sci_rule_ent = entropy(lm, sci_rules[0])
print(sci_rule_ent)

9.25324957176561


In [134]:
other_ent = list(map(lambda x: entropy(lm, x), other_rules))
print(sum(filter(lambda x: x != -1, other_ent)) / len(list(filter(lambda x: x != -1, other_ent))))


    

In [ ]:
lm.entropy()

In [19]:
len(other_ent)

97

In [20]:
len(sci_ent)

27

In [17]:
other_ent = list(map(lambda x: entropy(lm, x), other))
sci_ent = list(map(lambda x: entropy(lm, x), sci))

In [18]:
other_ent = list(filter(lambda x: x != -1, other_ent))
sci_ent = list(filter(lambda x: x != -1, sci_ent))

print(sum(other_ent)/len(other_ent))
print(sum(sci_ent)/len(sci_ent))


10.156344771039358
11.069099772871676


In [37]:
print('Posts')
print('----------------------------------')
print('Number:', len(df_posts_science))
print('avg char length of title:', df_posts_science.title.apply(len).mean(), 'std:', df_posts_science.title.apply(len).std()) 
print('avg char length of text:', df_posts_science.selftext.apply(len).mean(), 'std:',df_posts_science.selftext.apply(len).std()) 
print('avg char length of full text:', df_posts_science.fulltext.apply(len).mean(), 'std:',df_posts_science.fulltext.apply(len).std())
print('----------------------------------')
print('Comments')
print('----------------------------------')
print('Number:', len(df_comments_science))
print('avg char length of text:', df_comments_science.fillna(' ').body.apply(len).mean(), 'std:', df_comments_science.fillna(' ').body.apply(len).std()) 

Posts
----------------------------------
Number: 42740
avg char length of title: 83.963734206832 std: 64.08626776023108
avg char length of text: 17.540617688348153 std: 210.14624261062576
avg char length of full text: 101.50435189518016 std: 228.7489694447391
----------------------------------
Comments
----------------------------------
Number: 592813
avg char length of text: 271.2417625794306 std: 368.91475879038404


# Scratch

In [17]:
science_author_counts = pd.read_csv('../cleaned/sub_author_counts/science_author_counts_2017.csv')
science_comments = pd.read_csv('../cleaned/sub_comments/science_comments_2017.csv')

In [43]:
TEST_MONTHS = list(range(1,2))
from nltk.lm.smoothing import GoodTuring, KneserNey

In [46]:
##################################################################
# randomly sampling 200 users  
# defined as users with at least 5 COMMENTS ONLY in the respective community and month.
##################################################################
def get_active_users(author_df, month, author_col, threshold=5, num_authors=200):
#     if comments_only:
#         df_author_comments = author_df[author_df['kind'] == 'comment']
#         return df_author_comments[df_author_comments[month] > threshold].sample(num_authors)[author_col]
#     else:
    return author_df[author_df[month] > threshold].drop_duplicates().sample(num_authors)[author_col]
    
##################################################################   
# For each of these 200 active users
# select 5 random 10-word spans from 5 unique comments
# ASSUMING: from that month
# ASSUMING: this is one 10 word span from each comment, not 5 for each comment
##################################################################
def get_random_span(text, length):
    text = word_tokenize(text)
    print(len(text))
    try: 
        beg = random.randint(0, len(text) - length)
        end = beg + 10
        return t[beg:end]
    except:
        raise IndexError("Error: index out of range, probably happened if you didn't clean comments to be at least 10 words long")

def get_train_active_user_comments(df, authors, month, month_col='created_month'):
    df_month = df[df[month_col] == month]
    df_month_author = df_month[df_month['author'].apply(lambda x: x in authors)]
    df_grouped = df_month_author.groupby('author')
    sampled_comments = []
    for a, g in df_grouped:
        sample = g.sample(5)['body'].apply(lambda x: word_tokenize(x)[:10]) #TODO - change to random span
        sampled_comments.extend(sample)
    return sampled_comments

##################################################################
# To ensure robustness and maximize data efficiency, 
# we construct 100 SLMs for each community-month pair that has enough data
##################################################################
def construct_LM(text, vocab=None):
    if vocab:
        train, _ =  padded_everygram_pipeline(2, text)
    else: 
        train, vocab =  padded_everygram_pipeline(2, text)
    smoothing = GoodTuring(vocab)
    lm = InterpolatedLanguageModel(smoothing, 2)
#     lm = Laplace(2)
    lm.fit(train, vocab)
    return lm

# returns dict of {month:SLM}
def build_monthly_SLM(df, author_counts, slm_count):
    slm_dict = {}
    for m in TEST_MONTHS:
        print('Creating ', colored(str(slm_count) + ' SLMs ', 'red'), 'for', colored(' month ' + str(m), 'green'), '.....', end=' ')
        slms = []
        for i in range(1, slm_count + 1):
            active_users = get_active_users(author_counts, str(m), 'author')
            active_user_comments = get_train_active_user_comments(df, list(active_users), m)
            slm = construct_LM(active_user_comments)
            slms.append(slm)
        slm_dict[m] = slms
        print('Done')
    return slm_dict



In [47]:
test_slms = build_monthly_SLM(science_comments, science_author_counts, 1 )

Creating  1 SLMs  for  month 1 ..... 

TypeError: Can't instantiate abstract class GoodTuring with abstract methods alpha_gamma

In [38]:
text = word_tokenize(science_comments.loc[1].body)[:10]
test_text = list(ngrams(text, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))

In [40]:
test_slms[1][0].entropy(test_text)

9.951041003047091